In [1]:
#install dependencies

!pip install PyPDF2
!pip install pdfminer
!pip install pdfminer.six
!pip install tabulate
!pip install openpyxl

   ---------------------------------------- 0.0/232.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/232.6 kB ? eta -:--:--
   ------ -------------------------------- 41.0/232.6 kB 653.6 kB/s eta 0:00:01
   ---------------------------- ----------- 163.8/232.6 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 232.6/232.6 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.2 MB 325.1 kB/s eta 0:00:13
      --------------------------------------- 0.1/4.2 MB 751.6 kB/s eta 0:00:06
     -- ------------------------------------- 0.3/4.2 MB 1.9 MB/s eta 0:00:03
     ----- ---------------------------------- 0.6/4.2 MB 2.7 MB/s eta 0:00:02
     -------- ------------------------------- 0.9/4.2 MB 3.5 MB/s eta 0:00:01
     --------------- ------------------------ 1.6/4.2 MB 5.4 MB/s eta 0:00:01
   

I've transformed it into a txt file to then structure and get the output for each company - I now plan to create a table where I get the data in a more understandabel format that I can use for my analysis

In [49]:
#pdf is a download from a webiste where 560 different hydrogen companies are listed

from pdfminer.high_level import extract_text

def extract_text_from_pdf(pdf_path):
    text = extract_text(pdf_path)
    return text

pdf_path = 'pdf/deutschland_projekte.pdf'
text = extract_text_from_pdf(pdf_path)

with open('deutschland.txt', 'w') as file:
    file.write(text)


In [ ]:
import re

def clean_text(input_text):
    # Define patterns to match URLs, dates, and "1/95"
    url_pattern = r'https?://\S+'
    date_pattern = r'\d{1,2}/\d{1,2}/\d{2}, \d{1,2}:\d{1,2} (?:AM|PM)'
    pattern_1_95 = r'^\s*(?:[1-9]|[1-8][0-9]|9[0-5])/95\s*$'

    # Remove lines containing "Document", URLs, dates, and "1/95"
    cleaned_text = []
    previous_line_was_empty = False
    for line in input_text.split('\n'):
        if not re.search(r'^\s*Document\s*$', line) \
           and not re.search(url_pattern, line) \
           and not re.search(date_pattern, line) \
           and not re.search(pattern_1_95, line):
            # Remove double empty lines
            if line.strip() == '' and previous_line_was_empty:
                continue
            cleaned_text.append(line)
            previous_line_was_empty = line.strip() == ''
    
    return '\n'.join(cleaned_text)

# Read text from file
with open('new.txt', 'r') as file:
    text = file.read()

# Clean the text
cleaned_text = clean_text(text)

# Write the cleaned text back to the file
with open('deutschland_neu.txt', 'w') as file:
    file.write(cleaned_text)



In [46]:
from tabulate import tabulate
import pandas as pd


def parse_projects(text):
    projects = []
    current_project = {}
    current_key = None

    # Split the text into lines
    lines = text.split('\n')

    # Iterate over each line
    for line in lines:
        line = line.strip()
        # Skip empty lines
        if not line:
            if current_key is None:
                current_key = 'Address'
            elif current_key == 'Address':
                current_key = 'Category'
            elif current_key == 'Category':
                projects.append(current_project)
                current_project = {}
                current_key = None
        if line:
            if current_key is None:
                current_project['Name'] = line
            elif current_key == 'Address':
                current_project.setdefault('Address', []).append(line)
            elif current_key == 'Category':
                current_project.setdefault('Category', []).append(line)
            
    # Append the last project
    if current_project:
        projects.append(current_project)
    return projects

def format_projects(projects):
    formatted_projects = []
    for project in projects:
        formatted_project = {}
        formatted_project['Name'] = project['Name']
        # Join the lines of the address with a comma
        formatted_project['Address'] = ', '.join(project.get('Address', []))
        formatted_project['Category'] = project['Category']
        formatted_projects.append(formatted_project)
    return formatted_projects

def main():
    # Read text from file
    with open('data/cleaned_de.txt', 'r') as file:
        text = file.read()

    # Parse the projects
    projects = parse_projects(text)

    # Print projects as table
    df = pd.DataFrame(projects)
    df.to_excel('data/projects_de.xlsx', index=False)
    # Print the DataFrame
    
if __name__ == "__main__":
    main()
    

In [48]:
df = pd.read_excel('data/projects_de.xlsx')

df['Zip Code'] = df['Address'].str.split(', ').str[-2]
df['Country'] = df['Address'].str.split(', ').str[-1]

# Replace the symbols '[', ']', and ''' with an empty string in all columns
df = df.applymap(lambda x: str(x).replace('[', '').replace(']', '').replace("'", ''))

# Now 'projects' DataFrame will have those symbols removed from all columns

# Displaying the resulting DataFrame
df
df.to_excel('data/projects_de.xlsx', index=False)

In [32]:
with open('data/cleaned_de.txt', 'r') as file:
    lines = file.readlines()
    
for line in lines:
    print(line)

1s1 Energy Portugal, Unipessoal, Lda



Av. Duque de Loulé 5 6ºC

Lisbon, 1050-085

Portugal



Project

Member



ABB Ltd



Bruggerstrasse 66, 68

Baden, 5400

Switzerland



Industry

Corporate



ABS Hellenic



Sachtouri 1

Kallithea, Athens, 17674

Greece



Project

Member



ACCIONAPLUG S.L.



Avenida de Europa 10, PE La Moraleja

Alcobendas, Madrid, 28108

Spain



Industry

Corporate



ACTEMIUM NDT E&S



6 Rue Zamenhof

BOURG DE PEAGE, 26303

France



Project

Member



Advent Technologies



16 Panepistimiou St. 6th Floor

Athens, 10672

Greece



Industry

Corporate



AeH2



Ronda de Poniente,15, 1º Int. Izda. Tres Cantos

Madrid, 28760

Spain



EU National

Association



Aéroports de Lyon



BP 113

Lyon-Saint Exupéry Aéroport, 69125

France



Project

Member



AFC Energy



Unit 71.4 Dunsfold Park, Stovolds Hill

Cranleigh, GU6 8TB

United Kingdom



Industry

Corporate



African Hydrogen Partnership



Accuvise Administrators Limited, 7A Mayer Street

Port Lou